In [ ]:
# !pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio===0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

In [1]:
from IPython import display

def imshow(img):
    """ndarray 配列をインラインで Notebook 上に表示する。
    """
    ret, encoded = cv2.imencode(".jpg", img)
    display.display(display.Image(encoded))

In [8]:
import glob
import os

files = glob.glob('data/*.jpg')
img_re = os.path.join(os.path.dirname(files[0]),'mask_'+os.path.basename(files[0]))
print(img_re)

data\mask_000003.jpg


In [ ]:

import cv2
import numpy as np
import torch
import torchvision
from PIL import Image
from torchvision import models as models
from torchvision import transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PERSON_ID = 1

# モデルを作成する。
model = models.detection.maskrcnn_resnet50_fpn(pretrained=True).to(device)
model.eval()

# 画像を読み込む。

for file in files:
    img = Image.open(file)
    img_tensor = transforms.functional.to_tensor(img).to(device)

    # 推論する。
    output = model([img_tensor])[0]

    # ラベル 人 のマスクを取り出す。
    person_scores = output["masks"][output["labels"] == PERSON_ID, 0]
    person_score = person_scores.max(dim=0)[0]

    mask = person_score >= 0.7
    mask = mask.cpu().detach().numpy()

    # アルファチャンネル追加
    dst = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGRA)


    # mask の値が0のピクセルは透過する。
    dst[mask == 0] = 0
    imshow(dst)
#     fname = os.path.join(os.path.dirname(file),'mask_'+os.path.basename(file))
#     cv2.imwrite(fname, dst)